# Data Wrangling

## Librerias

### Instalacion

### Importacion

In [23]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans

## Carga Datos

In [3]:
dfFinalSet = pd.read_csv('FinalSet.csv', dtype = {'codigoencabezado': 'int32'})

dfFinalSet.describe(include = 'all')

,codigoencabezado,idregistro,linkpagina,fecharegistro,administracion,amueblado,antiguedad,banos,habitaciones,estudio,...,moneda,tipo,tipo_vendedor,favoritos,titulo,fecha_creacion,valido_hasta,descripcion,partner_code,user_id
count,2089.000000,2.089000e+03,2089,2089,441.000000,1263,1000,2089.000000,2083.000000,2083,...,2089,2088,1735,2089.000000,2089,2089,2089,2089,287,2.089000e+03
unique,NaN,NaN,1171,8,NaN,2,7,NaN,NaN,2,...,2,4,2,NaN,1089,1238,1160,1159,180,NaN
top,NaN,NaN,https://www.olx.com.gt/item/apartamento-en-ven...,2020-06-10,NaN,No,Hasta 5 años,NaN,NaN,No,...,US$,Apartamento,Inmobiliaria,NaN,APARTAMENTO EN ZONA 15 / ACODI,2020-05-14T17:23:53-06:00,2020-08-12T17:56:10-06:00,"Fácil acceso por Villa Nueva, Petapa, Villa Ca...",PAA-041-01-19-V,NaN
freq,NaN,NaN,8,550,NaN,974,309,NaN,NaN,2076,...,1335,2051,1454,NaN,10,8,8,10,5,NaN
mean,1045.573001,1.095163e+09,NaN,NaN,938.292517,NaN,NaN,1.955960,2.269323,NaN,...,NaN,NaN,NaN,2.089995,NaN,NaN,NaN,NaN,NaN,3.953937e+07
std,603.610614,1.528355e+07,NaN,NaN,5122.267271,NaN,NaN,0.866011,0.808192,NaN,...,NaN,NaN,NaN,14.648158,NaN,NaN,NaN,NaN,NaN,2.466658e+07
min,1.000000,9.637780e+08,NaN,NaN,0.000000,NaN,NaN,0.000000,1.000000,NaN,...,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,4.263970e+06
25%,523.000000,1.100240e+09,NaN,NaN,0.000000,NaN,NaN,1.000000,2.000000,NaN,...,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,2.401211e+07
50%,1046.000000,1.100368e+09,NaN,NaN,0.000000,NaN,NaN,2.000000,2.000000,NaN,...,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,2.745098e+07
75%,1568.000000,1.100389e+09,NaN,NaN,700.000000,NaN,NaN,2.000000,3.000000,NaN,...,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,6.162143e+07


## Data Wrangling

En esta seccion prepararemos el set de datos que se utilizara para realizar la construccion del modelo de regresion.

### Registros Unicos

Obtendremos solo los valores mas actualizados de los registros. Utilizaremos el atributo **idregistro** como identificador unico de los registros.

In [24]:
# Conversion a string del campo idregistro
dfAnalisisSet = dfFinalSet.astype({'idregistro': 'str'})

# Conversion a fecha del campo fecharegsitro
dfAnalisisSet['fecharegistro'] = pd.to_datetime(dfAnalisisSet['fecharegistro'], format = "%Y-%m-%d")

# Obtencion de fecha maxima por cada idregistro y filtro de esos registros.
dfAnalisisSet = dfAnalisisSet.loc[dfAnalisisSet.reset_index().groupby(['idregistro'])['fecharegistro'].idxmax()]

# Verificamos la cantidad de registros faltantes
dfAnalisisSet.shape

(1166, 26)

### Columna Precio Real

Procedemos a crear una columna llamada **precio_real** para establecer el precio en quetzales. Para los registros que tienen un precio en dolares se aplicara el factor de conversion a quetzales.

In [25]:
# Factor de conversion
cambio_moneda = 7.69

# Creacion de nueva columna
dfAnalisisSet['precio_real'] = np.where(dfAnalisisSet['moneda'] == 'US$', dfAnalisisSet['precio'] * cambio_moneda, 
                                        dfAnalisisSet['precio'])

# Visualizams los valores de precio real
dfAnalisisSet['precio_real']

1630    2299310.00
974     1207330.00
975      922800.00
965        6000.00
556        9612.50
           ...    
2044     697636.80
1974      19225.00
2043       4106.46
1973     753620.00
1645        900.00
Name: precio_real, Length: 1166, dtype: float64

### Columna Oferta

Procederemos a crear una columna llamada oferta para determinar si es una venta o alquiler de apartamento. Para ello vamos a especificar un valor pivote en la columna precio. Todo valor superior al valor pivote sera considerado como una venta.

In [26]:
# Asignacion de valor pivote
intValorPivote = 50000

# Creacion de nueva columna oferta.
dfAnalisisSet['oferta'] = ['Venta' if precio > intValorPivote else 'Alquiler' for precio in dfAnalisisSet['precio_real']]

# Verificamos cuantos registros quedaron en el dataset
dfAnalisisSet.groupby(['oferta']).count()['codigoencabezado']

oferta
Alquiler    730
Venta       436
Name: codigoencabezado, dtype: int64

### Filtro Inicial

Los primeros filtros a aplicar a nuestro set de datos son los siguientes:
* Tipo de oferta como Venta, ya que solo nos interesa realizar el modelo para apartamentos en venta.
* Amueblado con valor No, ya que segun el analisis para las ventas no hay data para este atributo.
* El tipo se especificara como "Apartamento" ya que es la clasificacion que posee mayor cantidad de registros.

In [27]:
# Especificamos los filtros
dfFiltradoSet = dfAnalisisSet[(dfAnalisisSet['oferta'] == 'Venta') & 
                              ((dfAnalisisSet['amueblado'] == 'No') | (dfAnalisisSet['amueblado'].isnull())) &
                             (dfAnalisisSet['tipo'] == 'Apartamento')]

# Verificamos las dimensiones
dfFiltradoSet.shape

(425, 28)

Ahora especificamos las columnas que utilizaremos en principio para la construccion del set de datos

In [28]:
# Seleccionando las columnas a utilizar
dfFiltradoSet = dfFiltradoSet[['idregistro', 'banos', 'habitaciones', 
                               'espacio_m2', 'latitud', 'longitud', 'moneda',
                               'parqueo', 'tipo_vendedor', 'precio_real']]

dfFiltradoSet.shape

(425, 10)

### Ingenieria de Variables

Se procedera a crear una nueva variable llamada ubicacion que sera el numero de cluster asignado dependiendo de los valores de longitud y latitud.

In [30]:
# Seleccion de numero de clusters.
# Nota: En Notebook Data Analysis podriamos identificar la cantidad de cluster por medio de elbow plot
intKClusters = 10

# Se obtienen los valores que se utilizaran para la construccion de modelo KMeans
npUbicaciones = np.array(list(zip(dfFiltradoSet['longitud'], 
                                  dfFiltradoSet['latitud']))).reshape(len(dfFiltradoSet['longitud']), 2)

# Construccion del modelo
kmeanModel = KMeans(n_clusters = intKClusters, max_iter = 2000).fit(npUbicaciones)

# Asignacion de clusters a registros
dfFiltradoSet['ubicacion'] = kmeanModel.labels_

# Verificacion de la nueva columna
dfFiltradoSet.head(10)

,idregistro,banos,habitaciones,espacio_m2,latitud,longitud,moneda,parqueo,tipo_vendedor,precio_real,ubicacion
1630,1007688385,3,3.0,232.0,14.591,-90.506,US$,NaN,Inmobiliaria,2299310.00,2
974,1014321966,2,1.0,78.0,14.600,-90.511,US$,Si,Inmobiliaria,1207330.00,2
975,1014324725,2,2.0,65.0,14.593,-90.491,US$,Si,Inmobiliaria,922800.00,2
557,1023944274,2,2.0,74.0,14.588,-90.503,US$,Si,Inmobiliaria,1647474.84,2
559,1029720345,2,3.0,290.0,14.607,-90.470,US$,Si,Inmobiliaria,5190750.00,5
564,1030164820,2,2.0,74.0,14.588,-90.503,US$,Si,Inmobiliaria,1647474.84,2
521,1039739280,2,2.0,74.0,14.588,-90.503,US$,Si,Inmobiliaria,1647474.84,2
526,1041192203,1,1.0,79.0,14.588,-90.503,Q,Si,Inmobiliaria,1329277.00,2
527,1041192824,2,2.0,79.0,14.588,-90.503,Q,Si,Inmobiliaria,1604456.00,2
528,1041193829,2,3.0,114.0,14.588,-90.503,Q,Si,Inmobiliaria,1604456.00,2


### Manejo Missing Values

Verificaremos para cada variable del dataframe la cantidad de valores faltantes.

In [31]:
# Verificacion de valores nulos en todas las columnas 
dfFiltradoSet.isnull().sum()

idregistro        0
banos             0
habitaciones      0
espacio_m2        0
latitud           0
longitud          0
moneda            0
parqueo          11
tipo_vendedor    97
precio_real       0
ubicacion         0
dtype: int64

Verifiquemos los rangos de valores y sus respectivas frecuencias.

In [32]:
pd.Series(dfFiltradoSet['ubicacion']).value_counts()

2    206
5     77
6     73
7     20
4     15
0     12
8     10
1      8
9      3
3      1
Name: ubicacion, dtype: int64